In [1]:
# load tidy dataset

import pandas as pd
import os.path
import numpy as np
from itertools import combinations
import glob
import sys
import time
from sklearn import svm
from sklearn.model_selection import train_test_split

## to increase the cell width of the notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# from sklearn.learning_curve import learning_curve
# from sklearn.grid_search import GridSearchCV
# from sklearn.cross_validation import ShuffleSplit
  

#import numpy as np
#import loadTidyCSV.py

def loadTidy(tidyData): 
    assert os.path.isfile(tidyData), "desired file does not exist" 
    df = pd.read_csv(tidyData, header = None, names = ["date", "animal", "session", "trial", "stimulus", "neuronID", "timePt", "CaSignal"])
    return df 

# helper functions to return the number of distinct types in the provided data frame
getNumNeurons = lambda df: len(np.unique(df['neuronID'].tolist()))
getNeurons = lambda df: np.unique(df['neuronID'].tolist())

getNumTrials = lambda df: len(np.unique(df['trial'].tolist()))
getTrials = lambda df: np.unique(df['trial'].tolist())


# pass in pre-filtered data set containing data for only one animal and the same session (ie SAME NEURONS!)
def getListsOfTrialIDs(df_animalSession):
    ## get trials for both stimuli
    df_anmlSessStimA = df_animalSession[df_animalSession['stimulus'] == stimA]
    df_anmlSessStimB = df_animalSession[df_animalSession['stimulus'] == stimB]
    print(stimA,stimB)

    ## get lists of trial numbers of each stimuli's presentations 
    trials_stimA = np.unique(df_anmlSessStimA['trial'].tolist())
    trials_stimB = np.unique(df_anmlSessStimB['trial'].tolist())
    print("trial IDs for each stimulus type",trials_stimA,trials_stimB)
    return (trials_stimA,trials_stimB)

## pass in a data frame with only a single animal and session    
def getNumTimePtsPerTrial(df_animalSession, trials_stimA, trials_stimB):

    #### get number of timePts in each trial selected above 
    ## (1 to 3 presentations of the same stimuli exist per session in Prabhat's data)
    numTimePtsPerTrial = np.empty((2,max(len(trials_stimA),len(trials_stimB))))
    numTimePtsPerTrial[:] = np.nan
    stimInd = 0;
    for thisStimTypeTrialNums in [trials_stimA, trials_stimB]:
        trialInd = 0
        for trial in thisStimTypeTrialNums:
            inds_thisTrial = (df_animalSession['trial']==trial)
            tmp_df_thisTrial = df_animalSession[inds_thisTrial] # gives all time points for all neurons
            numNeurons = getNumNeurons(tmp_df_thisTrial) 
            numTimePtsPerTrial[stimInd,trialInd] = np.sum(inds_thisTrial)/numNeurons 
            trialInd += 1
        stimInd += 1
    print(numTimePtsPerTrial) # rows are for stimuli type; cols are presentation of that stimulus
    return numTimePtsPerTrial

## test candidate comparisons based on whether the number of trials per session and approximate number of timePts match
def areNumTrialsPerStimulusEqual(numTimePtsPerTrial):
    
    ## no trials of either type --> discard this comparison for this animal/session   
    if np.all(np.isnan(numTimePtsPerTrial)):
        print("DISCARDED: neither stimulus type were found for this animal and session")
        return False  # skip to next session (WORK: handle this)
        
    ## different numbers of trials per stimuli/session --> discard this comparison for this animal/session 
    elif np.any(np.isnan(numTimePtsPerTrial)): 
        print("DISCARDED: mismatching numbers of trials per stimulus type for this animal/session")
        return False # skip to next session (WORK: handle this)

    ## FULFILLED here: condition that allows analysis to proceed to attempted data
    elif not np.any(np.isnan(numTimePtsPerTrial)): 
        print("trial numbers match")
    else:
        raise RuntimeError('unexpected trial comparison occurred')
        return False
    
    print("checking approx num of time points")

## input argument generated from getNumTimePtsPerTrial
def areNumTimePtsPerTrialSimilar(numTimePtsPerTrial):
    minTPs, maxTPs, meanTPs, stdTPs = timePtStats(numTimePtsPerTrial)             
    if (np.abs(minTPs-meanTPs) > (threshTPs_stdFromMean * np.abs(meanTPs-stdTPs))) \
        or (np.abs(maxTPs-meanTPs) > (threshTPs_stdFromMean * np.abs(meanTPs-stdTPs))): 
        print("DISCARDED: variance in trial length is above the user's threshold")
        return False # skip to next session (WORK: handle this)

    ### passed all criteria if it made it this far
    return True

## input argument created by getNumTimePtsPerTrial function
def timePtStats(numTimePtsPerTrial):
    minTPs = int(np.amin(numTimePtsPerTrial))
    maxTPs = int(np.amax(numTimePtsPerTrial))
    meanTPs = np.mean(numTimePtsPerTrial)
    stdTPs = np.std(numTimePtsPerTrial)

    ## useful for debugging
#     print('min', minTPs)
#     print('max', maxTPs)
#     print('std', stdTPs)
#     print('mean',meanTPs)
#     print('|min-mean|=',np.abs(minTPs-meanTPs))
#     print('|max-mean|=',np.abs(maxTPs-meanTPs))
#     print('|mean-std|=',np.abs(meanTPs-stdTPs))
#     print('thresh * |mean-std|=',(threshTPs_stdFromMean * np.abs(meanTPs-stdTPs)))
    
    return minTPs, maxTPs, meanTPs, stdTPs

def sameNeuronConcat(df_trunc, minTPs):
    neuronArr_anmlSess_stimA = np.empty((getNumNeurons(df_trunc),minTPs-1)) # -1 for 0 indexing
    neuronArr_anmlSess_stimB = np.empty((getNumNeurons(df_trunc),minTPs-1)) # -1 for 0 indexing
    for stimLst in [trials_stimA, trials_stimB]:
        if np.array_equal(stimLst,trials_stimA) == True:
            print('\nstimulus:', stimA)
        elif np.array_equal(stimLst,trials_stimB) == True:
            print('\nstimulus:', stimB)
        else:
            raise RuntimeError('unexpected trial concatenation condition occurred')
        print('selected minTPs: ', minTPs)

        ## create temporary sub matrix of concatenated cells for ONE stimulus
        tmp_neuronsArr_sameStim = np.empty((getNumNeurons(df_trunc),minTPs)) # for 0 indexing
        for trial in stimLst:
            print("appending same neurons in trial: ", trial)

            ##  create temporary sub matrix of same trial all cells
            tmp_neuronsArr_sameStim_sameTrial = np.empty((getNumNeurons(df_trunc),minTPs))
            for neuron in getNeurons(df_trunc):
                tmp_neuronInds = (df_trunc['trial']==trial) & (df_trunc['neuronID']==neuron)
                tmp_neuronSeries = df_trunc.loc[tmp_neuronInds,'CaSignal']
                tmp_neuronVec = tmp_neuronSeries.as_matrix()
                tmp_neuronsArr_sameStim_sameTrial[neuron-1,:] = tmp_neuronVec

            ## append trials to right of same stim if not the first entry
            if trial == stimLst[0]:
                tmp_neuronsArr_sameStim = np.copy(tmp_neuronsArr_sameStim_sameTrial)
            else:    
                tmp_neuronsArr_sameStim = np.concatenate((tmp_neuronsArr_sameStim, tmp_neuronsArr_sameStim_sameTrial), axis=1)
            print('same stim:', np.shape(tmp_neuronsArr_sameStim))

        ## save concatenated data to output variables
        if np.array_equal(stimLst,trials_stimA):
            neuronArr_anmlSess_stimA = tmp_neuronsArr_sameStim
        elif np.array_equal(stimLst,trials_stimB):
            neuronArr_anmlSess_stimB = tmp_neuronsArr_sameStim
        else:
            raise RuntimeError('unexpected same neuron concatenation state occured')
        
    return neuronArr_anmlSess_stimA, neuronArr_anmlSess_stimB

In [2]:
################ concatenate all .csv files exported from matlab into single pandas dataframe df

# tidy csv file and dir (use makeTidy_Anderson.m to convert Ann's structure to csv)
tidyDataDir = '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/'
tidyDataFileTemplate = 'mouse'
tidyDataFileExt = '.csv'

# get all input files you want to add to the same dataset
dataFiles = np.sort(glob.glob(\
    "/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse*.csv"))
print("data files: \n", dataFiles)


print("\n loading and appending to prior pandas data frame")
dataLst = []
for file in dataFiles:
    print(file)
    dataLst.append(loadTidy(file))
df = pd.concat(dataLst)

print('finished loading')

############# ALL DATA STORED HERE IN DF
    

data files: 
 [ '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse1.csv'
 '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse2.csv'
 '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse3.csv'
 '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse4.csv'
 '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse5.csv']

 loading and appending to prior pandas data frame
/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse1.csv
/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse2.csv
/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse3.csv
/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAn

In [3]:
## survey/search data to prepare for split operation
metaStrs = [['dates','date'],['animals','animal'],['sessions','session'],['maxTrials','trial'],['stimuli','stimulus']]

print("searching over entire data set to get range of various IDs for data (used in subsequent loops)") 

## this dictionary holds useful info regarding the range of inputs to loop/search over subsequently
metaDct = {}
for a,b in metaStrs:
    print(a,b)
    metaDct[a] = np.unique(df[b].tolist())
print(metaDct)


searching over entire data set to get range of various IDs for data (used in subsequent loops)
dates date
animals animal
sessions session
maxTrials trial
stimuli stimulus
{'dates': array(['2017_05_00'], 
      dtype='<U10'), 'animals': array([1, 3, 4, 5, 7]), 'sessions': array([1, 2, 3]), 'maxTrials': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21]), 'stimuli': array(['USS', 'baseline', 'female', 'male', 'mineral oil odor',
       'peanut odor', 'pred odor', 'rat', 'tone', 'toy'], 
      dtype='<U16')}


In [4]:
######### USER PARAMETERS #########
threshTPs_stdFromMean = 0.70

In [6]:
# get all pairs of stimuli
stimCmbTpl = tuple(combinations(metaDct['stimuli'],2)) 


######### MAIN LOOP ##########
df_SVM = pd.DataFrame(columns=('dateOfAnalysis', 'dateOfExperiment', 'animal', 'session', 'stimulusA', 'stimulusB', 'SVM_accuracy'))
ind_comparison = 0;
for (stimA, stimB) in stimCmbTpl:
    print((stimA,stimB))
    
    ## get all data for both trial types
    indsBoth = (df['stimulus']==stimA) | (df['stimulus']==stimB)
    df_bothStimuli = df[indsBoth]

    #### select data by animals and sessions
    for animal in metaDct['animals']:
        print("stimuli comparison num: ", ind_comparison+1)
        print('animal: ', animal)
        for session in metaDct['sessions']:
            print('session:', session)
            
            ## return subselection of data where the same neurons were recorded
            inds_animalSession = (df_bothStimuli['animal'] == animal) & (df_bothStimuli['session'] == session)
            df_animalSession = df_bothStimuli[inds_animalSession]
            try: 
                dateOfExperiment = df_animalSession['date'].values[0]
            except:
                dateOfExperiment = '?'
            print('date of exp:', dateOfExperiment)
            
            # get lists of trial IDs matching stimuli
            trials_stimA, trials_stimB = getListsOfTrialIDs(df_animalSession)
            
            #### skip this comparison <-- if the data don't match in number of trials 
            numTimePtsPerTrial = getNumTimePtsPerTrial(df_animalSession,trials_stimA,trials_stimB)
            if areNumTrialsPerStimulusEqual(numTimePtsPerTrial)==False:
                break     
                
            #### skip this comparison <-- if the data don't match in approx number of timePts
            if areNumTimePtsPerTrialSimilar(numTimePtsPerTrial) == False:
                break
                
            ## truncate longer trials to shortest trial and save to new df
            minTPs, maxTPs, meanTPs, stdTPs = timePtStats(numTimePtsPerTrial)
            truncLst = []
            for trial in np.concatenate((trials_stimA,trials_stimB)):

                ## equivalent to .loc
                # tmp_inds_trunc = (df_animalSession['trial']==trial) & (df_animalSession['timePt'] <= minTPs)
                # tmp_df_trunc = df_animalSession[tmp_inds_trunc]
                tmp_df_trunc = df_animalSession.loc[(df_animalSession['trial']==trial) & (df_animalSession['timePt'] <= minTPs), :]
                truncLst.append(tmp_df_trunc)
            df_trunc = pd.concat(truncLst)
                    
            #### concatenate same cells 
            ### loop over and concatenate neurons into the same row if they're the same neuron and stimuli 
            ##      (ie mouse, session, stimuli)
            neuronArr_anmlSess_stimA, neuronArr_anmlSess_stimB = sameNeuronConcat(df_trunc,minTPs)
            print(np.shape(neuronArr_anmlSess_stimA), np.shape(neuronArr_anmlSess_stimB))
            
            ######### SVM #########
            
            ## create SVM format input by concatenating both classes (stimuli types); y is the labels
            print("stimA, stimB",np.shape(neuronArr_anmlSess_stimA), np.shape(neuronArr_anmlSess_stimB))
            X = np.concatenate((neuronArr_anmlSess_stimA, neuronArr_anmlSess_stimB), axis = 0)
            y = np.empty((neuronArr_anmlSess_stimA.shape[0]+neuronArr_anmlSess_stimB.shape[0]))
            y[:neuronArr_anmlSess_stimA.shape[0]] = 0
            y[neuronArr_anmlSess_stimB.shape[0]:] = 1
            print("X:", X.shape)
            print("y:", y.shape)
            
            print("k fold partitioning")
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
            
            print("training SVM")
            clf = svm.SVC(kernel='linear')
            clf.fit(X_train, y_train)
            
            print("testing SVM")
            tmp_score = clf.score(X_test, y_test)
            print(tmp_score)
            
            analysisDate = pd.to_datetime('now')
            
            df_SVM.loc[ind_comparison] = [analysisDate, dateOfExperiment, animal, session, stimA, stimB, tmp_score]
            print(df_SVM)

            ind_comparison += 1
            
            #             tmp_SVMresult = pd.DataFrame({"animal": [animal]})#,{"testAccuracy": tmp_score})
#             tmp_SVMresult = pd.DataFrame([animal], columns = list([1])) #,{"testAccuracy": tmp_score})
#             df_SVM.loc[df_SVM.index.max() + 1] = [animal, session, tmp_score]
#             df_SVM.loc[totalNumComparisons] = [animal, session, tmp_score]
#             df_SVM.loc[totalNumComparisons] = [0,1,2]
#             df_SVM = pd.DataFrame({'mouse': animal, 'session': session, 'SVMaccuracy': tmp_score}, index =totalNumComparisons)
#             df_SVM.iloc[1] = dict(x=9, y=99)
#             df_SVM.append(tmp_SVMresult, ignore_index=True)
#             clf.predict(X_test, y_test)
            
    
    ##########33
#             cv = ShuffleSplit(X_train.shape[0], n_iter=10, test_size=0.2, random_state=0)
            
            # WORK: optional gridsearch
#             gammas = np.logspace(-6, -1, 10)
#             classifier = GridSearchCV(estimator=estimator, cv=cv, param_grid=dict(gamma=gammas))
#             classifier.fit(X_train, y_train)
            
#             title = 'Learning Curves (SVM, linear kernel, $\gamma=%.6f$)' %classifier.best_estimator_.gamma
#             estimator = SVC(kernel='linear', gamma=classifier.best_estimator_.gamma)
#             plot_learning_curve(estimator, title, X_train, y_train, cv=cv)
#             plt.show()
                
            print('\n')
        print('########\n')

print('total number of comparisons: ', ind_comparison+1)

('USS', 'baseline')
stimuli comparison num:  1
animal:  1
session: 1
date of exp: 2017_05_00
USS baseline
trial IDs for each stimulus type [ 5 10 16] [ 1  8 15]
[[ 1117.  1116.  1117.]
 [  370.   363.  1476.]]
trial numbers match
checking approx num of time points
DISCARDED: variance in trial length is above the user's threshold
########

stimuli comparison num:  1
animal:  3
session: 1
date of exp: 2017_05_00
USS baseline
trial IDs for each stimulus type [ 4 11 17] [ 1  8 15]
[[ 1267.  1127.  1123.]
 [  674.   682.   678.]]
trial numbers match
checking approx num of time points

stimulus: USS
selected minTPs:  674
appending same neurons in trial:  4
same stim: (30, 674)
appending same neurons in trial:  11
same stim: (30, 1348)
appending same neurons in trial:  17
same stim: (30, 2022)

stimulus: baseline
selected minTPs:  674
appending same neurons in trial:  1
same stim: (30, 674)
appending same neurons in trial:  8
same stim: (30, 1348)
appending same neurons in trial:  15
same sti


stimulus: USS
selected minTPs:  1114
appending same neurons in trial:  4
same stim: (30, 1114)
appending same neurons in trial:  11
same stim: (30, 2228)
appending same neurons in trial:  17
same stim: (30, 3342)

stimulus: female
selected minTPs:  1114
appending same neurons in trial:  7
same stim: (30, 1114)
appending same neurons in trial:  9
same stim: (30, 2228)
appending same neurons in trial:  19
same stim: (30, 3342)
(30, 3342) (30, 3342)
stimA, stimB (30, 3342) (30, 3342)
X: (60, 3342)
y: (60,)
k fold partitioning
training SVM
testing SVM
0.666666666667
       dateOfAnalysis dateOfExperiment animal session stimulusA stimulusB  \
0 2017-07-03 18:49:27       2017_05_00      3       1       USS  baseline   
1 2017-07-03 18:49:28       2017_05_00      3       2       USS  baseline   
2 2017-07-03 18:49:29       2017_05_00      3       3       USS  baseline   
3 2017-07-03 18:49:32       2017_05_00      1       1       USS    female   
4 2017-07-03 18:49:32       2017_05_00      1

appending same neurons in trial:  6
same stim: (21, 1114)
appending same neurons in trial:  13
same stim: (21, 2228)
appending same neurons in trial:  16
same stim: (21, 3342)

stimulus: male
selected minTPs:  1114
appending same neurons in trial:  4
same stim: (21, 1114)
appending same neurons in trial:  11
same stim: (21, 2228)
appending same neurons in trial:  19
same stim: (21, 3342)
(21, 3342) (21, 3342)
stimA, stimB (21, 3342) (21, 3342)
X: (42, 3342)
y: (42,)
k fold partitioning
training SVM
testing SVM
0.666666666667
        dateOfAnalysis dateOfExperiment animal session stimulusA stimulusB  \
0  2017-07-03 18:49:27       2017_05_00      3       1       USS  baseline   
1  2017-07-03 18:49:28       2017_05_00      3       2       USS  baseline   
2  2017-07-03 18:49:29       2017_05_00      3       3       USS  baseline   
3  2017-07-03 18:49:32       2017_05_00      1       1       USS    female   
4  2017-07-03 18:49:32       2017_05_00      1       2       USS    female   
5


stimulus: USS
selected minTPs:  1113
appending same neurons in trial:  4
same stim: (56, 1113)
appending same neurons in trial:  11
same stim: (56, 2226)

stimulus: male
selected minTPs:  1113
appending same neurons in trial:  6
same stim: (56, 1113)
appending same neurons in trial:  14
same stim: (56, 2226)
(56, 2226) (56, 2226)
stimA, stimB (56, 2226) (56, 2226)
X: (112, 2226)
y: (112,)
k fold partitioning
training SVM
testing SVM
0.739130434783
        dateOfAnalysis dateOfExperiment animal session stimulusA stimulusB  \
0  2017-07-03 18:49:27       2017_05_00      3       1       USS  baseline   
1  2017-07-03 18:49:28       2017_05_00      3       2       USS  baseline   
2  2017-07-03 18:49:29       2017_05_00      3       3       USS  baseline   
3  2017-07-03 18:49:32       2017_05_00      1       1       USS    female   
4  2017-07-03 18:49:32       2017_05_00      1       2       USS    female   
5  2017-07-03 18:49:33       2017_05_00      1       3       USS    female   
6

same stim: (35, 1109)

stimulus: male
selected minTPs:  1109
appending same neurons in trial:  7
same stim: (35, 1109)
(35, 1109) (35, 1109)
stimA, stimB (35, 1109) (35, 1109)
X: (70, 1109)
y: (70,)
k fold partitioning
training SVM
testing SVM
0.928571428571
        dateOfAnalysis dateOfExperiment animal session stimulusA stimulusB  \
0  2017-07-03 18:49:27       2017_05_00      3       1       USS  baseline   
1  2017-07-03 18:49:28       2017_05_00      3       2       USS  baseline   
2  2017-07-03 18:49:29       2017_05_00      3       3       USS  baseline   
3  2017-07-03 18:49:32       2017_05_00      1       1       USS    female   
4  2017-07-03 18:49:32       2017_05_00      1       2       USS    female   
5  2017-07-03 18:49:33       2017_05_00      1       3       USS    female   
6  2017-07-03 18:49:34       2017_05_00      3       1       USS    female   
7  2017-07-03 18:49:34       2017_05_00      3       2       USS    female   
8  2017-07-03 18:49:35       2017_05_00

...


stimulus: tone
selected minTPs:  979
appending same neurons in trial:  6
same stim: (34, 979)
appending same neurons in trial:  11
same stim: (34, 1958)
appending same neurons in trial:  19
same stim: (34, 2937)

stimulus: toy
selected minTPs:  979
appending same neurons in trial:  2
same stim: (34, 979)
appending same neurons in trial:  12
same stim: (34, 1958)
appending same neurons in trial:  20
same stim: (34, 2937)
(34, 2937) (34, 2937)
stimA, stimB (34, 2937) (34, 2937)
X: (68, 2937)
y: (68,)
k fold partitioning
training SVM
testing SVM
0.642857142857
         dateOfAnalysis dateOfExperiment animal session stimulusA  \
0   2017-07-03 18:49:27       2017_05_00      3       1       USS   
1   2017-07-03 18:49:28       2017_05_00      3       2       USS   
2   2017-07-03 18:49:29       2017_05_00      3       3       USS   
3   2017-07-03 18:49:32       2017_05_00      1       1       USS   
4   2017-07-03 18:49:32       2017_05_00      1       2       USS   
5   2017-07-03 18:49:


stimulus: tone
selected minTPs:  1115
appending same neurons in trial:  7
same stim: (49, 1115)
appending same neurons in trial:  12
same stim: (49, 2230)

stimulus: toy
selected minTPs:  1115
appending same neurons in trial:  2
same stim: (49, 1115)
appending same neurons in trial:  15
same stim: (49, 2230)
(49, 2230) (49, 2230)
stimA, stimB (49, 2230) (49, 2230)
X: (98, 2230)
y: (98,)
k fold partitioning
training SVM
testing SVM
0.95
         dateOfAnalysis dateOfExperiment animal session stimulusA  \
0   2017-07-03 18:49:27       2017_05_00      3       1       USS   
1   2017-07-03 18:49:28       2017_05_00      3       2       USS   
2   2017-07-03 18:49:29       2017_05_00      3       3       USS   
3   2017-07-03 18:49:32       2017_05_00      1       1       USS   
4   2017-07-03 18:49:32       2017_05_00      1       2       USS   
5   2017-07-03 18:49:33       2017_05_00      1       3       USS   
6   2017-07-03 18:49:34       2017_05_00      3       1       USS   
7   201


stimulus: tone
selected minTPs:  1120
appending same neurons in trial:  1
same stim: (36, 1120)
appending same neurons in trial:  12
same stim: (36, 2240)

stimulus: toy
selected minTPs:  1120
appending same neurons in trial:  5
same stim: (36, 1120)
appending same neurons in trial:  14
same stim: (36, 2240)
(36, 2240) (36, 2240)
stimA, stimB (36, 2240) (36, 2240)
X: (72, 2240)
y: (72,)
k fold partitioning
training SVM
testing SVM
0.8
         dateOfAnalysis dateOfExperiment animal session stimulusA  \
0   2017-07-03 18:49:27       2017_05_00      3       1       USS   
1   2017-07-03 18:49:28       2017_05_00      3       2       USS   
2   2017-07-03 18:49:29       2017_05_00      3       3       USS   
3   2017-07-03 18:49:32       2017_05_00      1       1       USS   
4   2017-07-03 18:49:32       2017_05_00      1       2       USS   
5   2017-07-03 18:49:33       2017_05_00      1       3       USS   
6   2017-07-03 18:49:34       2017_05_00      3       1       USS   
7   2017

same stim: (20, 1115)
appending same neurons in trial:  9
same stim: (20, 2230)

stimulus: toy
selected minTPs:  1115
appending same neurons in trial:  6
same stim: (20, 1115)
appending same neurons in trial:  12
same stim: (20, 2230)
(20, 2230) (20, 2230)
stimA, stimB (20, 2230) (20, 2230)
X: (40, 2230)
y: (40,)
k fold partitioning
training SVM
testing SVM
0.625
         dateOfAnalysis dateOfExperiment animal session stimulusA  \
0   2017-07-03 18:49:27       2017_05_00      3       1       USS   
1   2017-07-03 18:49:28       2017_05_00      3       2       USS   
2   2017-07-03 18:49:29       2017_05_00      3       3       USS   
3   2017-07-03 18:49:32       2017_05_00      1       1       USS   
4   2017-07-03 18:49:32       2017_05_00      1       2       USS   
5   2017-07-03 18:49:33       2017_05_00      1       3       USS   
6   2017-07-03 18:49:34       2017_05_00      3       1       USS   
7   2017-07-03 18:49:34       2017_05_00      3       2       USS   
8   2017-07-0

tone toy
trial IDs for each stimulus type [ 2 15] [ 6 10]
[[ 1023.   922.]
 [ 1016.  1029.]]
trial numbers match
checking approx num of time points

stimulus: tone
selected minTPs:  922
appending same neurons in trial:  2
same stim: (40, 922)
appending same neurons in trial:  15
same stim: (40, 1844)

stimulus: toy
selected minTPs:  922
appending same neurons in trial:  6
same stim: (40, 922)
appending same neurons in trial:  10
same stim: (40, 1844)
(40, 1844) (40, 1844)
stimA, stimB (40, 1844) (40, 1844)
X: (80, 1844)
y: (80,)
k fold partitioning
training SVM
testing SVM
0.8125
         dateOfAnalysis dateOfExperiment animal session stimulusA  \
0   2017-07-03 18:49:27       2017_05_00      3       1       USS   
1   2017-07-03 18:49:28       2017_05_00      3       2       USS   
2   2017-07-03 18:49:29       2017_05_00      3       3       USS   
3   2017-07-03 18:49:32       2017_05_00      1       1       USS   
4   2017-07-03 18:49:32       2017_05_00      1       2       USS   

In [7]:
## save raw svm (animal/session) results to /gitTracked/python/
timestr = time.strftime("%Y%m%d-%H%M%S")
print(timestr)

# Write out raw analysis to csv file
OUTPUT_FILENAME = 'SVM_analysis_raw' + timestr + '.csv'
df_SVM.to_csv(OUTPUT_FILENAME, header=True, index=False)

20170703-131300


In [39]:
## save summary svm data

# svmGrouped, df_SVM_summary = df_SVM.groupby(['stimulusA', 'stimulusB'], as_index=False)

# f = {'SVM_accuracy':['sum','mean'], 'B':['prod']}

stimGrouped = df_SVM.groupby(['stimulusA','stimulusB'], as_index=True)
df_SVM_summaryDesc = stimGrouped.describe()

print(grouped.describe())      

# Write out raw analysis to csv file
OUTPUT_FILENAME = 'SVM_analysis_summary' + timestr + '.csv'
df_SVM_summaryDesc.to_csv(OUTPUT_FILENAME, header=True, index=True)

                                  SVM_accuracy                                \
                                         count      mean       std       min   
stimulusA        stimulusB                                                     
USS              baseline                  3.0  0.746032  0.068732  0.666667   
                 female                    6.0  0.729365  0.099085  0.600000   
                 male                     15.0  0.750156  0.138517  0.500000   
                 mineral oil odor          9.0  0.762600  0.176724  0.375000   
                 peanut odor               7.0  0.811494  0.078156  0.684211   
                 pred odor                 7.0  0.797647  0.155758  0.578947   
                 rat                      13.0  0.748175  0.126804  0.400000   
                 tone                     15.0  0.677386  0.182696  0.300000   
                 toy                      15.0  0.756293  0.116296  0.500000   
baseline         female                 

In [34]:
## inspect grouped-by object created above

for key, item in stimGrouped:
    print(grouped.get_group(key), "\n\n")

   SVM_accuracy animal      dateOfAnalysis dateOfExperiment session
0      0.666667      3 2017-07-03 18:49:27       2017_05_00       1
1      0.785714      3 2017-07-03 18:49:28       2017_05_00       2
2      0.785714      3 2017-07-03 18:49:29       2017_05_00       3 


   SVM_accuracy animal      dateOfAnalysis dateOfExperiment session
3      0.800000      1 2017-07-03 18:49:32       2017_05_00       1
4      0.600000      1 2017-07-03 18:49:32       2017_05_00       2
5      0.666667      1 2017-07-03 18:49:33       2017_05_00       3
6      0.666667      3 2017-07-03 18:49:34       2017_05_00       1
7      0.857143      3 2017-07-03 18:49:34       2017_05_00       2
8      0.785714      3 2017-07-03 18:49:35       2017_05_00       3 


    SVM_accuracy animal      dateOfAnalysis dateOfExperiment session
9       0.700000      1 2017-07-03 18:49:38       2017_05_00       1
10      0.700000      1 2017-07-03 18:49:39       2017_05_00       2
11      0.666667      1 2017-07-03 18:4

In [ ]:
ratData = df.loc[df['stimulusType'] == 'rat',:]
ussData = df.loc[df['stimulusType'] == 'USS',:]

print(ussData)



df.corr()


# Rename the impact force column
df = df.rename(columns={'impact force (mN)': 'impf'})

